####NTIS data crawling

In [1]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import urllib
import pandas as pd
print("stage1 processing")
#페이지수 자동 가져오기
url = "https://www.ntis.go.kr/ThSearchProjectList.do?gubun=link&searchWord=%EB%B9%88%EA%B3%A4&searchType=&oldSearchWord=%EB%B9%88%EA%B3%A4&resultSearch=&pageNumber=1&ssoKnfSlct=&ascDesc=ASC&useYn=N&oldQuery=%EB%B9%88%EA%B3%A4&oldAddQuery=&dbt=project&checkYn=&searchOption16=&pageYn=&technologyClassification=ST&directorySearchYear=&directorySearchOption1=&directorySearchOption2=&directorySearchOption3=&downloadTarget=project&startRow=&endRow=&rqstPurpCd=&infoPrctuseDes=&layerChoice=&sort=RANK%2FDESC&pageSize=10&navigation1%5B0%5D=2019&navigation1%5B1%5D=2018&navigation1%5B2%5D=2017&navigation1%5B3%5D=2016&navigation1%5B4%5D=2015"
r=requests.get(url)
c=r.content
soup = BeautifulSoup(c,'html.parser')
paging = soup.find("div",{"id":"pageing"}).find_all("a")
start_page=paging[0].attrs['href']
last_page=paging[len(paging)-1].attrs['href']
print("stage2 processing")
#URL 갯수 가져와서 자동 URL 생성 & 각 url 에서 콘텐츠만 추출
front_url = "https://www.ntis.go.kr/ThSearchProjectList.do?gubun=link&searchWord=%EB%B9%88%EA%B3%A4&searchType=&oldSearchWord=%EB%B9%88%EA%B3%A4&resultSearch=&pageNumber="
back_url = "&ssoKnfSlct=&ascDesc=ASC&useYn=N&oldQuery=%EB%B9%88%EA%B3%A4&oldAddQuery=%26SN04%3D2019%7C2015%7C2017%7C2018%7C2016&dbt=project&checkYn=&searchOption16=&pageYn=Y&technologyClassification=ST&directorySearchYear=&directorySearchOption1=&directorySearchOption2=&directorySearchOption3=&downloadTarget=project&startRow=&endRow=&rqstPurpCd=&infoPrctuseDes=&layerChoice=&sort=RANK%2FDESC&pageSize=10&navigation1%5B0%5D=2019&navigation1%5B1%5D=2018&navigation1%5B2%5D=2017&navigation1%5B3%5D=2016&navigation1%5B4%5D=2015"
web_contentlist=[]
for i in range(int(start_page),int(last_page)+1): 
  url = front_url+str(i)+back_url
  r = requests.get(url).content
  web_contentlist.append(BeautifulSoup(r, "html.parser"))
print("stage3 processing")
#mdata클래스 안에 있는 데이터만 다시 정리
data_list = []
for i in range(0, int(last_page)):
  c = web_contentlist[i].find_all(class_='mdata')
  for x in c:
    p = x.text.replace("\n","").split("/")
    data_list.append(p)
print("stage4 processing")
#데이터 프레임 생성해서 dictionary looping 한 후 데이터로 저장 
output = pd.DataFrame()
dicts=dict.fromkeys(['ID','Year', 'class','PM', 'operated_by','executor','amount'])
for i in range(len(data_list)):
  dicts["ID"] = data_list[i][0]
  dicts["Year"] = data_list[i][1]
  dicts["class"] = data_list[i][2]
  dicts["PM"] = data_list[i][3]
  dicts["operated_by"] = data_list[i][4]
  dicts["executor"] = data_list[i][5]
  dicts["amount"] = data_list[i][6]
  output=output.append(dicts, ignore_index=True)
print("ended")

stage1 processing
stage2 processing
stage3 processing
stage4 processing
ended
